IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Zoé REY

**Student Number**: 400677239

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [55]:
import pandas as pd

df = pd.read_csv('/content/Nata.csv', sep=';')

# Drop rows with missing values
df.dropna(inplace=True)

display(df.head())

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,09/04/2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,03/08/2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,02/10/2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [56]:
constant_columns = df.columns[df.nunique() == 1]

print(f"Columns with constant values: {list(constant_columns)}")

# Drop constant columns from my DataFrame
df = df.drop(columns=constant_columns)

print(f"Number of columns remaining: {df.shape[1]}")

Columns with constant values: ['Z_CostContact', 'Z_Revenue']
Number of columns remaining: 27


In [57]:
from datetime import datetime

df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], format='mixed', dayfirst=True, errors='coerce')

# Drop rows where Dt_Customer could not be parsed and resulted in NaT
df.dropna(subset=['Dt_Customer'], inplace=True)

today = datetime.today()

df["Dt_Customer"] = (today - df["Dt_Customer"]).dt.days

df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,4980,58,635,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,4134,38,11,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,4481,26,426,...,2,10,4,0,0,0,0,0,0,0
3,6182,1984,Graduation,Together,26646.0,1,0,4074,26,11,...,0,4,6,0,0,0,0,0,0,0
4,5324,1981,PhD,Married,58293.0,1,0,4330,94,173,...,3,6,5,0,0,0,0,0,0,0


(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [58]:
from sklearn.model_selection import train_test_split

# Define target variable (y)
y = df['MntWines']

# Define features (X) by dropping ID, Dt_Customer, MntWines, and Response
X = df.drop(columns=['ID', 'Dt_Customer', 'MntWines', 'Response'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1772, 23)
X_test shape: (444, 23)
y_train shape: (1772,)
y_test shape: (444,)


(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [68]:
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=6,
    min_samples_leaf=4,
    random_state=20
)

wine_spending_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', rf_model)
])

wine_spending_pipeline.fit(X_train, y_train)

print("Wine spending prediction pipeline created and fitted successfully!")

Wine spending prediction pipeline created and fitted successfully!


In [69]:
new_customer_features_raw = {
    'Age': 48,
    'Education': 'Graduation',
    'Marital_Status': 'Married',
    'Income': 80000.0,
    'Kidhome': 1,
    'Teenhome': 1,
    'Dt_Customer_str': '2016-10-10',
    'Recency': 43,
    'NumDealsPurchases': 2,
    'NumWebPurchases': 1,
    'NumCatalogPurchases': 0,
    'NumStorePurchases': 15,
    'NumWebVisitsMonth': 5,
    'AcceptedCmp1': 0,
    'AcceptedCmp2': 0,
    'AcceptedCmp3': 0,
    'AcceptedCmp4': 0,
    'AcceptedCmp5': 0,
    'Complain': 0
}

new_customer_df = pd.DataFrame([new_customer_features_raw])

reference_year = 2014 # Based on the comment 'This will be 2014'
new_customer_df['Year_Birth'] = reference_year - new_customer_df['Age']

new_customer_df = new_customer_df.drop(columns=['Age', 'Dt_Customer_str'])

missing_mnt_cols = [
    'MntFruits', 'MntMeatProducts', 'MntFishProducts',
    'MntSweetProducts', 'MntGoldProds'
]
for col in missing_mnt_cols:
    if col not in new_customer_df.columns:
        new_customer_df[col] = X_train[col].mean()

new_customer_processed = new_customer_df[X_train.columns]

predicted_spending_wines = wine_spending_pipeline.predict(new_customer_processed)

print(f"Predicted spending on wines for the new customer: {predicted_spending_wines[0]:.2f}")

Predicted spending on wines for the new customer: 368.25


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [70]:
y_pred = wine_spending_pipeline.predict(X_test)

r2 = r2_score(y_test, y_pred)

mae = mean_absolute_error(y_test, y_pred)

print(f"Model Performance on Test Data:")
print(f"R-squared: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

print("\n--- Model Performance Description ---")
if r2 > 0.7:
    print("The model demonstrates strong predictive power, explaining a large proportion of the variance in wine spending.")
elif r2 > 0.5:
    print("The model shows moderate predictive power, explaining a fair amount of the variance in wine spending.")
else:
    print("The model's predictive power is relatively low, explaining a limited proportion of the variance in wine spending.")

print(f"On average, the model's predictions for wine spending are off by approximately ${mae:.2f}.")

Model Performance on Test Data:
R-squared: 0.7714
Mean Absolute Error (MAE): 91.01

--- Model Performance Description ---
The model demonstrates strong predictive power, explaining a large proportion of the variance in wine spending.
On average, the model's predictions for wine spending are off by approximately $91.01.


(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [71]:
cv_scores = cross_val_score(wine_spending_pipeline, X_train, y_train, cv=6, scoring='r2')

print(f"Cross-validation R-squared scores: {cv_scores}")
print(f"Mean R-squared across 6 folds: {cv_scores.mean():.4f}")
print(f"Standard deviation of R-squared across 6 folds: {cv_scores.std():.4f}")

Cross-validation R-squared scores: [0.7979451  0.77729687 0.81476282 0.78527942 0.79624085 0.79017444]
Mean R-squared across 6 folds: 0.7936
Standard deviation of R-squared across 6 folds: 0.0117


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [86]:
param_grid = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [5, 10, 15],
    'model__min_samples_split': [3, 6],
    'model__min_samples_leaf': [2, 4, 8]
}

grid_search = GridSearchCV(estimator=wine_spending_pipeline, param_grid=param_grid,
                           cv=6, scoring='r2', n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print("Best hyperparameters found:", grid_search.best_params_)
print("Best R-squared score:", grid_search.best_score_)

best_rf_model = grid_search.best_estimator_
print("Best estimator retrieved successfully!")

Fitting 6 folds for each of 36 candidates, totalling 216 fits
Best hyperparameters found: {'model__max_depth': 15, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3, 'model__n_estimators': 100}
Best R-squared score: 0.8039105686290039
Best estimator retrieved successfully!


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [75]:
categorical_features_lr = X_train.select_dtypes(include=['object']).columns

lr_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_lr)
    ],
    remainder='passthrough'
)

lr_model = LinearRegression()

lr_pipeline = Pipeline(steps=[
    ('preprocessor', lr_preprocessor),
    ('regressor', lr_model)
])

lr_pipeline.fit(X_train, y_train)

print("Linear Regression model built and fitted successfully!")

y_pred_lr = lr_pipeline.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print("\n--- Linear Regression Model Performance on Test Data ---")
print(f"R-squared (Linear Regression): {r2_lr:.4f}")
print(f"Mean Absolute Error (Linear Regression): {mae_lr:.2f}")

y_pred_rf_best = best_rf_model.predict(X_test)
r2_rf_best = r2_score(y_test, y_pred_rf_best)
mae_rf_best = mean_absolute_error(y_test, y_pred_rf_best)

print("\n--- Best Random Forest Model Performance on Test Data (from Q1.vi) ---")
print(f"R-squared (Best Random Forest): {r2_rf_best:.4f}")
print(f"Mean Absolute Error (Best Random Forest): {mae_rf_best:.2f}")

print("\n--- Comparison ---")
if r2_rf_best > r2_lr:
    print(f"The Best Random Forest Model has a higher R-squared ({r2_rf_best:.4f}) than the Linear Regression Model ({r2_lr:.4f}), indicating better explanatory power.")
else:
    print(f"The Linear Regression Model has a higher R-squared ({r2_lr:.4f}) than the Best Random Forest Model ({r2_rf_best:.4f}).")

if mae_rf_best < mae_lr:
    print(f"The Best Random Forest Model has a lower Mean Absolute Error ({mae_rf_best:.2f}) than the Linear Regression Model ({mae_lr:.2f}), suggesting more accurate predictions on average.")
else:
    print(f"The Linear Regression Model has a lower Mean Absolute Error ({mae_lr:.2f}) than the Best Random Forest Model ({mae_rf_best:.2f}).")

Linear Regression model built and fitted successfully!

--- Linear Regression Model Performance on Test Data ---
R-squared (Linear Regression): 0.6664
Mean Absolute Error (Linear Regression): 119.73

--- Best Random Forest Model Performance on Test Data (from Q1.vi) ---
R-squared (Best Random Forest): 0.7785
Mean Absolute Error (Best Random Forest): 87.56

--- Comparison ---
The Best Random Forest Model has a higher R-squared (0.7785) than the Linear Regression Model (0.6664), indicating better explanatory power.
The Best Random Forest Model has a lower Mean Absolute Error (87.56) than the Linear Regression Model (119.73), suggesting more accurate predictions on average.


(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [78]:
# Step 1: Compute the absolute residuals for both models
abs_residuals_lr = np.abs(y_pred_lr - y_test)
abs_residuals_rf = np.abs(y_pred_rf_best - y_test)

# Step 2: Create a DataFrame for plotting residuals
residuals_df = pd.DataFrame({
    'LR_Absolute_Residuals': abs_residuals_lr,
    'RF_Absolute_Residuals': abs_residuals_rf
})

fig = px.scatter(
    residuals_df,
    x='LR_Absolute_Residuals',
    y='RF_Absolute_Residuals',
    title='Comparison of Absolute Residuals (LR vs. RF)',
    labels={
        'LR_Absolute_Residuals': 'Linear Regression Absolute Residuals',
        'RF_Absolute_Residuals': 'Random Forest Absolute Residuals'
    },
    hover_data={'LR_Absolute_Residuals': ':.2f', 'RF_Absolute_Residuals': ':.2f'}
)

# Step 3: Add a 45 degree reference line to the plot
# Determine min/max values for the 45-degree line based on the data
min_val = min(residuals_df['LR_Absolute_Residuals'].min(), residuals_df['RF_Absolute_Residuals'].min())
max_val = max(residuals_df['LR_Absolute_Residuals'].max(), residuals_df['RF_Absolute_Residuals'].max())

buffer = (max_val - min_val) * 0.05
min_val -= buffer
max_val += buffer

fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="orange", width=2, dash="dash")
)

fig.update_layout(showlegend=False)
fig.show()

The plot compares the error of two models by placing the Linear Regression (LR) on the X-axis and the Random Forest (RF) on the Y-axis, with a dashed y=x reference line.

Since the vast majority of the data points fall below this 45-degree line, it indicates that the Y-value (RF error) is generally smaller than the X-value (LR error) for those data points.

This clustering below the line serves as compelling visual evidence that **the Random Forest model is superior, as it consistently produces a smaller magnitude of error across the dataset compared to the Linear Regression model**.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [80]:
meat_percentiles = df['MntMeatProducts'].quantile([1/3, 2/3])
lower_bound = meat_percentiles.iloc[0]
upper_bound = meat_percentiles.iloc[1]

print(f"33.33% percentile for MntMeatProducts: {lower_bound:.2f}")
print(f"66.67% percentile for MntMeatProducts: {upper_bound:.2f}")

def label_meat_spending(spending):
    if spending < lower_bound:
        return 'low'
    elif spending > upper_bound:
        return 'high'
    else:
        return 'medium'

df['Meat_Spending_Category'] = df['MntMeatProducts'].apply(label_meat_spending)

print("\nDistribution of Meat Spending Categories:")
print(df['Meat_Spending_Category'].value_counts())

print("\nDataFrame head with new 'Meat_Spending_Category' column:")
print(df[['MntMeatProducts', 'Meat_Spending_Category']].head())

33.33% percentile for MntMeatProducts: 24.00
66.67% percentile for MntMeatProducts: 146.33

Distribution of Meat Spending Categories:
Meat_Spending_Category
medium    746
high      739
low       731
Name: count, dtype: int64

DataFrame head with new 'Meat_Spending_Category' column:
   MntMeatProducts Meat_Spending_Category
0              546                   high
1                6                    low
2              127                 medium
3               20                    low
4              118                 medium


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [83]:
y_knn = df['Meat_Spending_Category']
X_knn = df.drop(columns=[
    'ID',
    'MntMeatProducts',
    'Meat_Spending_Category',
    'MntWines',
    'Response'
])

X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(
    X_knn, y_knn, test_size=0.2, random_state=20, stratify=y_knn
)

numerical_features_knn = X_knn.select_dtypes(include=np.number).columns.tolist()
categorical_features_knn = X_knn.select_dtypes(include='object').columns.tolist()

preprocessor_knn = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_knn),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_knn)
    ],
    remainder='passthrough'
)

knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_knn),
    ('classifier', KNeighborsClassifier())
])

knn_pipeline.fit(X_train_knn, y_train_knn)

print("K-nearest-neighbors model pipeline created and fitted successfully")

print(f"X_train_knn shape: {X_train_knn.shape}")
print(f"X_test_knn shape: {X_test_knn.shape}")
print(f"y_train_knn shape: {y_train_knn.shape}")
print(f"y_test_knn shape: {y_test_knn.shape}")

K-nearest-neighbors model pipeline created and fitted successfully!
X_train_knn shape: (1772, 23)
X_test_knn shape: (444, 23)
y_train_knn shape: (1772,)
y_test_knn shape: (444,)


(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [85]:
y_pred_knn = knn_pipeline.predict(X_test_knn)

report = classification_report(y_test_knn, y_pred_knn)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

        high       0.86      0.84      0.85       148
         low       0.79      0.91      0.84       146
      medium       0.72      0.63      0.67       150

    accuracy                           0.79       444
   macro avg       0.79      0.79      0.79       444
weighted avg       0.79      0.79      0.79       444



Based on the report, we can see how well the KNN model distinguishes between customers with 'low', 'medium', and 'high' meat spending habits:

* **Precision** measures the fraction of correct positive predictions out of all samples predicted as positive for a class (minimizing false positives).

* **Recall** measures the fraction of correct positive predictions out of all actual positive samples in that class (minimizing false negatives).

* **The F1-score **is the harmonic mean of precision and recall, serving as a single metric to summarize the balance between the two.

* **Support** simply indicates the total number of actual instances of each class in the dataset being evaluated.

* **Accuracy** provides the overall percentage of correctly classified instances across all classes.

* **The Macro avg** calculates the unweighted average of the metrics across all classes, treating them equally, while the Weighted avg uses the class Support to compute a more representative average when class imbalance is present.

Ultimately, the report details the K-Nearest Neighbors (KNN) model's proficiency in accurately classifying customers into their respective meat spending groups.


# Describe how you used Gen. AI. in this assignment (2 points)


I used Gen. AI. for this assignment to help me understand my errors and to adjust my code to solve them.



**Note**: The remaining 5 points will be assigned to readability of the work.